In [7]:
import time
import json
import csv
# Selenium의 기본 webdriver 대신 undetected_chromedriver를 uc 라는 이름으로 가져옵니다.
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# --- 1단계: 모든 상품의 고유 ID만 수집하는 함수 (탐지 우회 드라이버 사용) ---
def get_all_product_ids():
    """
    undetected-chromedriver를 사용하여 봇 탐지를 우회하고,
    스크롤을 통해 동적으로 로드되는 모든 상품의 ID를 수집합니다.
    """
    print("--- 1단계: 상품 ID 수집 시작 ---")

    options = uc.ChromeOptions()
    # options.add_argument("--headless") # uc 드라이버는 headless 모드에서 불안정할 수 있으므로, 초기 실행 시에는 비활성화를 권장합니다.

    # webdriver.Chrome(...) 대신 uc.Chrome(...)을 사용하여 드라이버를 생성합니다.
    driver = uc.Chrome(options=options)

    product_ids = []
    try:
        burger_page_url = "https://www.lotteeatz.com/brand/ria?categoryId=C1001"
        driver.get(burger_page_url)
        driver.maximize_window()

        print("✅ '탐지 우회 드라이버'로 페이지에 접속했습니다.")
        # 페이지의 각종 보안 스크립트가 실행될 시간을 넉넉하게 줍니다.
        time.sleep(3)

        # 팝업 제거 로직
        try:
            popup_close_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-close-popup")))
            popup_close_button.click()
            print("✅ 프로모션 팝업을 닫았습니다.")
        except TimeoutException:
            print("ℹ️ 프로모션 팝업이 없습니다.")

        print("⏳ Lazy Loading 컨텐츠를 불러오기 위해 페이지를 스크롤합니다...")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) # 스크롤 후 데이터가 렌더링될 물리적인 시간 확보

        print("⏳ 상품 목록의 최종 렌더링을 기다립니다...")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.prod-list > li[onclick*="goBrandDetail"]'))
        )
        print("✅ 상품 목록 렌더링 완료!")

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_list = soup.select('ul.prod-list > li')

        for item in product_list:
            onclick_attr = item.get('onclick')
            if onclick_attr and 'goBrandDetail' in onclick_attr:
                product_id = onclick_attr.split("'")[1]
                product_ids.append(product_id)

        if not product_ids:
             print("⚠️ 1단계: ID를 수집하지 못했습니다.")
        else:
            print(f"✅ 1단계: 총 {len(product_ids)}개의 상품 ID 수집을 완료했습니다.")

    except TimeoutException:
        print("❌ 1단계: 대기 시간 초과! 상품 목록을 찾는 데 실패했습니다.")
        driver.save_screenshot("uc_failure_screenshot.png")
        print("   'uc_failure_screenshot.png' 스크린샷을 확인하여 원인을 파악해보세요.")
    except Exception as e:
        print(f"❌ 1단계: ID 수집 중 예상치 못한 오류가 발생했습니다: {e}")
    finally:
        driver.quit()
        print("✅ 1단계: 브라우저를 종료합니다.")
        return product_ids

# --- 2단계: ID 리스트를 기반으로 상세 정보를 수집하는 함수 ---
def get_product_details(product_ids):
    """
    수집된 ID 리스트를 바탕으로 각 상품의 상세 페이지에 직접 접속하여
    이름, 영양정보, 알러지 정보, 원산지 정보를 수집합니다.
    """
    if not product_ids:
        print("⚠️ 2단계: 수집할 ID가 없어 작업을 종료합니다.")
        return []

    print("\n--- 2단계: 상세 정보 수집 시작 ---")

    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    all_products_data = []

    for i, pid in enumerate(product_ids):
        try:
            detail_url = f"https://www.lotteeatz.com/products/introductions/{pid}"
            driver.get(detail_url)

            detail_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-fold.detail-info")))
            driver.execute_script("arguments[0].click();", detail_button)

            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@class='btext-tit' and text()='영양소 정보']")))

            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

            name = detail_soup.select_one('div.prod-tit').text.strip()
            nutrition_info, allergy_info, origin_info = {}, "", {}

            nutrition_div = detail_soup.find('div', class_='btext-tit', string='영양소 정보')
            if nutrition_div:
                nutrition_table = nutrition_div.find_next_sibling('div', class_='tbl-info-wrap')
                if nutrition_table:
                    for row in nutrition_table.select('tbody > tr'):
                        key = row.select_one('th').text.strip()
                        value = row.select_one('td').text.strip()
                        nutrition_info[key] = value

            allergy_p = detail_soup.find('div', class_='btext-tit', string='알러지 정보')
            if allergy_p:
                allergy_info = allergy_p.find_next_sibling('p', class_='btext').text.strip()

            origin_div = detail_soup.find('div', class_='btext-tit', string='원산지 정보')
            if origin_div:
                origin_table = origin_div.find_next_sibling('div', class_='tbl-info-wrap')
                if origin_table:
                    for row in origin_table.select('tbody > tr'):
                        key = row.select_one('th').text.strip()
                        value = row.select_one('td').text.strip()
                        origin_info[key] = value

            product_data = {"id": pid, "name": name, "nutrition": nutrition_info, "allergy": allergy_info, "origin": origin_info}
            all_products_data.append(product_data)
            print(f"({i+1}/{len(product_ids)}) ✅ '{name}' 정보 수집 성공")

        except Exception as e:
            print(f"({i+1}/{len(product_ids)}) ❌ ID '{pid}' 처리 중 오류 발생: {e}")

    driver.quit()
    print("✅ 2단계: 브라우저를 종료합니다.")
    return all_products_data

# --- 3단계: 수집된 데이터를 파일로 저장하는 함수 ---
def save_data(data):
    """
    최종 수집된 데이터를 JSON 파일과 CSV 파일 두 가지 형태로 저장합니다.
    """
    if not data:
        print("⚠️ 3단계: 저장할 데이터가 없습니다.")
        return

    print("\n--- 3단계: 데이터 저장 시작 ---")

    # JSON 저장
    file_name_json = "lotte_eatz_burgers_final.json"
    with open(file_name_json, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ JSON 파일 저장 완료: {file_name_json}")

    # CSV 저장
    file_name_csv = "lotte_eatz_burgers_final.csv"
    csv_data = []
    for item in data:
        nutrition_str = ", ".join([f"{k}: {v}" for k, v in item.get('nutrition', {}).items()])
        origin_str = ", ".join([f"{k}: {v}" for k, v in item.get('origin', {}).items()])

        csv_data.append([
            item.get('id', ''),
            item.get('name', ''),
            nutrition_str,
            item.get('allergy', ''),
            origin_str
        ])

    with open(file_name_csv, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(['ID', '이름', '영양정보', '알러지정보', '원산지정보'])
        writer.writerows(csv_data)
    print(f"✅ CSV 파일 저장 완료: {file_name_csv}")

# --- 메인 실행 블록: 모든 단계를 순차적으로 실행 ---
if __name__ == '__main__':
    # 1단계 실행
    product_ids = get_all_product_ids()

    # 1단계가 성공적으로 ID를 반환했을 경우에만 2, 3단계 실행
    if product_ids:
        # 2단계 실행
        details = get_product_details(product_ids)

        # 2단계가 성공적으로 데이터를 반환했을 경우에만 3단계 실행
        if details:
            # 3단계 실행
            save_data(details)
            print("\n🎉 모든 크롤링 작업이 성공적으로 완료되었습니다! 🎉")
        else:
            print("\n최종적으로 수집된 상세 데이터가 없습니다.")
    else:
        print("\n최종적으로 수집된 ID가 없어 작업을 종료합니다.")

--- 1단계: 상품 ID 수집 시작 ---
✅ '탐지 우회 드라이버'로 페이지에 접속했습니다.
ℹ️ 프로모션 팝업이 없습니다.
⏳ Lazy Loading 컨텐츠를 불러오기 위해 페이지를 스크롤합니다...
⏳ 상품 목록의 최종 렌더링을 기다립니다...
❌ 1단계: 대기 시간 초과! 상품 목록을 찾는 데 실패했습니다.
   'uc_failure_screenshot.png' 스크린샷을 확인하여 원인을 파악해보세요.
✅ 1단계: 브라우저를 종료합니다.

최종적으로 수집된 ID가 없어 작업을 종료합니다.


In [3]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# --- '디버그 및 진단'에 초점을 맞춘 함수 ---
def diagnose_product_list_loading():
    print("--- 롯데잇츠 방어 로직 진단 시작 ---")

    chrome_options = Options()
    # chrome_options.add_argument("--headless") # 반드시 주석 처리하여 눈으로 확인
    chrome_options.add_argument("--start-maximized") # 최대화
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")

    # chromedriver.exe 파일이 코드와 같은 폴더에 있다고 가정
    service = ChromeService(executable_path='./chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=chrome_options)

    burger_page_url = "https://www.lotteeatz.com/brand/ria?categoryId=C1001"
    driver.get(burger_page_url)

    try:
        print("✅ 페이지 접속 완료. 초기 스크립트 로딩을 위해 5초 대기...")
        time.sleep(5) # 페이지의 모든 리소스(JS, CSS)가 로드될 시간을 넉넉하게 줌

        # 팝업 제거
        try:
            popup_close_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-close-popup")))
            popup_close_button.click()
            print("✅ 프로모션 팝업을 닫았습니다.")
        except TimeoutException:
            print("ℹ️ 프로모션 팝업이 없습니다.")

        print("⏳ 1차 시도: 페이지 최하단으로 스크롤하여 Lazy Loading을 유발합니다.")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print("...스크롤 완료. 데이터 렌더링을 위해 3초 추가 대기...")
        time.sleep(3)

        # 스크롤 후에도 목록이 보이지 않을 수 있으므로, '버거' 탭을 다시 클릭해봄
        print("⏳ 2차 시도: '버거' 탭을 다시 클릭하여 렌더링을 강제로 촉발합니다.")
        try:
            burger_tab = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '버거')]"))
            )
            driver.execute_script("arguments[0].click();", burger_tab)
            print("...클릭 완료. 데이터 렌더링을 위해 3초 추가 대기...")
            time.sleep(3)
        except Exception as e:
            print(f"⚠️ '버거' 탭 재클릭 실패: {e}")

        # 최종 확인
        print("⏳ 최종 확인: 상품 목록이 실제로 렌더링되었는지 10초간 기다립니다.")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.prod-list > li[onclick*="goBrandDetail"]'))
        )

        print("\n🎉🎉🎉 성공! 마침내 상품 목록을 찾아냈습니다! 🎉🎉🎉")
        # 성공했다면 10초간 화면을 보여주고 종료
        time.sleep(10)

    except TimeoutException:
        print("\n❌❌❌ 실패: 최종적으로 상품 목록을 찾는 데 실패했습니다. ❌❌❌")
        print("... 문제 파악을 위해 'failure_screenshot.png' 이름으로 스크린샷을 저장합니다.")
        driver.save_screenshot("failure_screenshot.png")
        print("... 브라우저를 닫지 않고 30초간 대기합니다. 화면을 직접 확인해주세요!")
        print("... [확인 사항] 눈으로는 상품 목록이 보이는데 스크립트가 못찾나요? 아니면 아예 빈 화면인가요?")
        time.sleep(30)
    finally:
        print("--- 진단 종료. 브라우저를 닫습니다. ---")
        driver.quit()


# --- 메인 실행 블록 ---
if __name__ == '__main__':
    diagnose_product_list_loading()

--- 롯데잇츠 방어 로직 진단 시작 ---
✅ 페이지 접속 완료. 초기 스크립트 로딩을 위해 5초 대기...
ℹ️ 프로모션 팝업이 없습니다.
⏳ 1차 시도: 페이지 최하단으로 스크롤하여 Lazy Loading을 유발합니다.
...스크롤 완료. 데이터 렌더링을 위해 3초 추가 대기...
⏳ 2차 시도: '버거' 탭을 다시 클릭하여 렌더링을 강제로 촉발합니다.
⚠️ '버거' 탭 재클릭 실패: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff71ffefca5+79861]
	GetHandleVerifier [0x0x7ff71ffefd00+79952]
	(No symbol) [0x0x7ff71fd6cada]
	(No symbol) [0x0x7ff71fdc4457]
	(No symbol) [0x0x7ff71fdc471c]
	(No symbol) [0x0x7ff71fe18217]
	(No symbol) [0x0x7ff71fdecb1f]
	(No symbol) [0x0x7ff71fe14f8b]
	(No symbol) [0x0x7ff71fdec8b3]
	(No symbol) [0x0x7ff71fdb5272]
	(No symbol) [0x0x7ff71fdb6043]
	GetHandleVerifier [0x0x7ff7202ab9dd+2946349]
	GetHandleVerifier [0x0x7ff7202a5c5a+2922410]
	GetHandleVerifier [0x0x7ff7202c59e7+3052855]
	GetHandleVerifier [0x0x7ff72000aa8e+189918]
	GetHandleVerifier [0x0x7ff720012a2f+222591]
	GetHandleVerifier [0x0x7ff71fff8ac4+116244]
	GetHandleVerifier [0x0x7ff71fff8c79+116681]
	GetHandleVerifier [0x0x7ff71ffdf058+11176]
	BaseThreadInitThunk

In [8]:
import time
import json
import csv
import undetected_chromedriver as uc
import requests  # requests 라이브러리 추가

def get_data_with_hybrid_approach():
    """
    [하이브리드 전략]
    1. uc-driver로 브라우저 세션을 열어 쿠키를 획득.
    2. requests에 그 쿠키를 이식하여 API를 직접 호출.
    """
    print("--- [최종 전략] 하이브리드 크롤링 시작 ---")

    # --- 1단계: Selenium으로 '진짜 사람'인 척 세션 및 쿠키 획득 ---
    print("⏳ 1단계: 탐지 우회 드라이버로 사이트에 접속하여 세션을 활성화합니다...")
    driver = uc.Chrome()
    all_products_data = []

    try:
        # 상세 페이지가 아닌, 가장 기본적인 메인 페이지에 접속하여 세션을 만듭니다.
        driver.get("https://www.lotteeatz.com/brand/ria")
        driver.maximize_window()
        # 사이트가 쿠키 및 세션 관련 스크립트를 실행할 시간을 넉넉히 줍니다.
        time.sleep(5)
        print("✅ 세션 활성화 완료. 브라우저로부터 쿠키를 추출합니다.")

        # Selenium이 활성화한 세션의 모든 쿠키를 가져옵니다.
        cookies = driver.get_cookies()

    except Exception as e:
        print(f"❌ 1단계(드라이버 실행) 실패: {e}")
        return None
    finally:
        driver.quit()
        print("✅ 1단계 완료: 쿠키 획득 후 브라우저를 종료했습니다.")


    # --- 2단계: 획득한 쿠키로 Requests를 통해 API 직접 호출 ---
    if not cookies:
        print("❌ 쿠키를 획득하지 못해 2단계를 진행할 수 없습니다.")
        return None

    print("\n⏳ 2단계: 획득한 쿠키를 사용하여 서버 API에 직접 데이터를 요청합니다...")

    # requests에서 사용할 세션 객체 생성
    session = requests.Session()

    # Selenium 쿠키를 requests 세션 쿠키로 변환하여 이식
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])

    # 이전에 확인했던 API 주소와 데이터
    api_url = "https://api.lotteeatz.com/api/v1/product/list"

    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Content-Type': 'application/json;charset=UTF-8',
        'Origin': 'https://www.lotteeatz.com',
        'Referer': 'https://www.lotteeatz.com/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }

    payload = {
        "brandCode": "RIA", "categoryId": "C1001", "dstrcCode": "",
        "menuClassCode": "100", "page": 1, "pageSize": 100,
        "searchType": "", "searchWord": "", "sortType": "RANK"
    }

    try:
        response = session.post(api_url, headers=headers, data=json.dumps(payload), timeout=10)
        response.raise_for_status() # 200번대 응답이 아니면 오류 발생

        print(f"✅ 2단계 성공! API로부터 데이터를 수신했습니다. (상태 코드: {response.status_code})")

        api_data = response.json()
        product_list = api_data.get('data', {}).get('list', [])

        if not product_list:
            print("⚠️ API 응답에 상품 목록이 없습니다.")
            return []

        # (선택) 2단계 코드와 동일하게 데이터 파싱
        print("\n--- 수집된 버거 메뉴 목록 ---")
        for i, product in enumerate(product_list):
             # 상세 정보까지는 없으므로, 일단 이름과 가격만 추출
            name = product.get('productName', '이름 없음')
            price = product.get('productPrice', 0)
            pid = product.get('productCode', '')
            print(f"{i+1:2d}. {name:<20} | {price: ,}원")
            # 추후 상세 정보가 필요하면 이 pid로 2단계 로직을 다시 만들 수 있습니다.
            all_products_data.append({"id": pid, "name": name, "price": price})

        return all_products_data

    except requests.exceptions.HTTPError as errh:
        print(f"❌ 2단계 실패(HTTP 오류): {errh}")
        print(f"   응답 내용: {response.text}")
        return None
    except Exception as e:
        print(f"❌ 2단계 실패(기타 오류): {e}")
        return None


# --- 3단계: 수집된 데이터를 파일로 저장하는 함수 ---
# (이전과 동일, price 필드만 추가된 것을 감안하여 수정)
def save_data(data):
    # ... 이전 3단계 코드와 거의 동일 ...
    # 필요시 이 부분은 이전 코드를 붙여넣어 사용하시면 됩니다.
    print(f"\n✅ 총 {len(data)}개의 상품 데이터를 JSON 파일로 저장합니다.")
    with open("lotte_eatz_burgers_hybrid_final.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("   저장 완료: lotte_eatz_burgers_hybrid_final.json")


# --- 메인 실행 블록 ---
if __name__ == '__main__':
    final_data = get_data_with_hybrid_approach()

    if final_data:
        save_data(final_data)
        print("\n🎉 모든 안티-크롤링을 우회하고 데이터 수집에 최종 성공했습니다! 🎉")
    else:
        print("\n최종적으로 모든 시도가 실패했습니다.")

--- [최종 전략] 하이브리드 크롤링 시작 ---
⏳ 1단계: 탐지 우회 드라이버로 사이트에 접속하여 세션을 활성화합니다...
✅ 세션 활성화 완료. 브라우저로부터 쿠키를 추출합니다.
✅ 1단계 완료: 쿠키 획득 후 브라우저를 종료했습니다.

⏳ 2단계: 획득한 쿠키를 사용하여 서버 API에 직접 데이터를 요청합니다...
❌ 2단계 실패(기타 오류): HTTPSConnectionPool(host='api.lotteeatz.com', port=443): Max retries exceeded with url: /api/v1/product/list (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000028AC5D06F90>, 'Connection to api.lotteeatz.com timed out. (connect timeout=10)'))

최종적으로 모든 시도가 실패했습니다.


In [10]:
import time
import json
import undetected_chromedriver as uc
import requests

def final_diagnosis_with_logging():
    print("--- [최종 진단] 모든 단계 로깅 시작 ---")

    # --- 1단계: Selenium으로 쿠키 획득 ---
    print("\n[ Phase 1: 쿠키 획득 ]")
    print("="*30)
    driver = None
    cookies = None
    try:
        print("  - Undetected-Chromedriver 인스턴스를 생성합니다.")
        driver = uc.Chrome()
        driver.get("https://www.lotteeatz.com/brand/ria")
        print(f"  - 페이지 접속 성공: {driver.current_url}")
        print("  - 세션 활성화를 위해 5초간 대기합니다...")
        time.sleep(5)

        cookies = driver.get_cookies()
        print(f"  - ✅ 쿠키 추출 성공! 총 {len(cookies)}개의 쿠키를 발견했습니다.")
        # 너무 길어질 수 있으니, 주요 쿠키 이름만 출력
        cookie_names = [cookie['name'] for cookie in cookies]
        print(f"  - 쿠키 이름 목록: {cookie_names}")

    except Exception as e:
        print(f"  - ❌ 1단계(드라이버 실행) 실패: {e}")
        return
    finally:
        if driver:
            driver.quit()
        print("  - 브라우저를 종료했습니다.")
    print("="*30)

    # --- 2단계: Requests로 API 호출 ---
    if not cookies:
        print("\n[ Phase 2: API 요청 ] - 쿠키가 없어 중단합니다.")
        return

    print("\n[ Phase 2: API 요청 ]")
    print("="*30)

    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])
    print("  - Requests 세션에 모든 쿠키를 성공적으로 이식했습니다.")

    api_url = "https://api.lotteeatz.com/api/v1/product/list"
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Content-Type': 'application/json;charset=UTF-8',
        'Origin': 'https://www.lotteeatz.com',
        'Referer': 'https://www.lotteeatz.com/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }
    payload = {
        "brandCode": "RIA", "categoryId": "C1001", "dstrcCode": "",
        "menuClassCode": "100", "page": 1, "pageSize": 100,
        "searchType": "", "searchWord": "", "sortType": "RANK"
    }

    print(f"  - API 타겟 URL: {api_url}")
    print(f"  - 전송할 헤더 정보:\n{json.dumps(headers, indent=4)}")
    print(f"  - 전송할 데이터(Payload):\n{json.dumps(payload, indent=4)}")

    try:
        print("\n  - 이제 서버에 POST 요청을 보냅니다. (Timeout=10초)")
        response = session.post(api_url, headers=headers, data=json.dumps(payload), timeout=10)

        print(f"  - ✅ 요청 성공! 서버로부터 응답을 받았습니다.")
        print(f"  - 상태 코드: {response.status_code}")
        response.raise_for_status()

        # 성공 시에만 데이터 처리
        api_data = response.json()
        print("  - 수신된 JSON 데이터 파싱 성공!")
        # ... 데이터 처리 로직 ...

    # 오류를 종류별로 더 세분화하여 기록
    except requests.exceptions.ConnectTimeout:
        print("  - ❌ [명백한 원인] ConnectTimeoutError 발생!")
        print("  - 서버가 우리의 초기 연결 시도 자체에 응답하지 않았습니다.")
        print("  - 이는 방화벽, IP 차단, 또는 TLS/SSL 지문 탐지에 의한 네트워크 레벨 차단일 가능성이 매우 높습니다.")
    except requests.exceptions.HTTPError as errh:
        print(f"  - ❌ HTTP 오류 발생: {errh}")
        print(f"  - 서버가 연결은 허용했으나, 요청 내용(헤더, 쿠키 등)에 문제가 있어 거부했습니다.")
        print(f"  - 응답 내용: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"  - ❌ 요청 관련 오류 발생: {e}")
    finally:
        print("="*30)


if __name__ == '__main__':
    final_diagnosis_with_logging()
    print("\n--- 모든 진단 프로세스 종료 ---")

--- [최종 진단] 모든 단계 로깅 시작 ---

[ Phase 1: 쿠키 획득 ]
  - Undetected-Chromedriver 인스턴스를 생성합니다.
  - 페이지 접속 성공: https://www.lotteeatz.com/brand/ria
  - 세션 활성화를 위해 5초간 대기합니다...
  - ✅ 쿠키 추출 성공! 총 6개의 쿠키를 발견했습니다.
  - 쿠키 이름 목록: ['_ga', '_ga_KP58JY4ZM5', 'nlbi_3152415', 'incap_ses_1571_3152415', 'visid_incap_3152415', 'JSESSIONID_app']
  - 브라우저를 종료했습니다.

[ Phase 2: API 요청 ]
  - Requests 세션에 모든 쿠키를 성공적으로 이식했습니다.
  - API 타겟 URL: https://api.lotteeatz.com/api/v1/product/list
  - 전송할 헤더 정보:
{
    "Accept": "application/json, text/plain, */*",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://www.lotteeatz.com",
    "Referer": "https://www.lotteeatz.com/",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}
  - 전송할 데이터(Payload):
{
    "brandCode": "RIA",
    "categoryId": "C1001",
    "dstrcCode": "",
    "menuClassCode": "100",
    "page": 1,
    "pageSize": 100,
    "searchType": "",
    "searc